#### Setup

In [1]:
import os, sys
PWD = os.getenv('PWD')
print(PWD)

/protwis/sites/protwis


In [2]:
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "protwis.settings")
import django
django.setup()

/env/lib/python3.4/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### Imports

In [3]:
from itertools import combinations
from tqdm import tqdm
import pandas as pd

from signprot.interactions import *
from protein.models import Protein, ProteinSegment, ProteinFamily
from signprot.models import SignprotComplex
from seqsign.sequence_signature import SequenceSignature

## Defining the protein sets

### G Proteins

In [4]:
sets = {
    'Gi/o': [],
    'Gq/11': [],
    'Gs': [],
    'G12/13': [],
}

set_combinations = list(combinations(sets.keys(), 2))
segments = list(ProteinSegment.objects.filter(proteinfamily='GPCR'))

In [5]:
set_combinations

[('G12/13', 'Gs'),
 ('G12/13', 'Gi/o'),
 ('G12/13', 'Gq/11'),
 ('Gs', 'Gi/o'),
 ('Gs', 'Gq/11'),
 ('Gi/o', 'Gq/11')]

### Querying all Complexes

In [6]:
rec_classes = ProteinFamily.objects.filter(parent=1).exclude(slug=100)

In [7]:
complexes = []
for pc in SignprotComplex.objects.all().prefetch_related('structure', 'protein'):
    t = {}
    
    t['gprot']  = pc.protein.family.name
    t['rec_pdb'] = pc.structure.protein_conformation.protein.entry_short()
    t['rec_obj'] = pc.structure.protein_conformation.protein
    t['rec_class'] = pc.structure.protein_conformation.protein.get_protein_class()
    t['rec_short'] = pc.structure.protein_conformation.protein.parent.entry_short()
    t['rec_name'] = pc.structure.protein_conformation.protein.parent.name
    
    complexes.append(t)

pd.DataFrame(complexes)

,gprot,rec_class,rec_name,rec_obj,rec_pdb,rec_short
0,Gs,Class B1 (Secretin),CT receptor,5uz7,5UZ7,CALCR
1,Gs,Class B1 (Secretin),CT receptor,6niy,6NIY,CALCR
2,Gs,Class B1 (Secretin),GLP-1 receptor,5vai,5VAI,G1SGD4
3,Gs,Class B1 (Secretin),GLP-1 receptor,6b3j,6B3J,GLP1R
4,Gs,Class A (Rhodopsin),A<sub>2A</sub> receptor,6gdg,6GDG,AA2AR
5,Gs,Class A (Rhodopsin),&beta;<sub>2</sub>-adrenoceptor,3sn6,3SN6,ADRB2
6,Gi/o,Class A (Rhodopsin),&mu; receptor,6dde,6DDE,OPRM
7,Gi/o,Class A (Rhodopsin),&mu; receptor,6ddf,6DDF,OPRM
8,Gi/o,Class A (Rhodopsin),Rhodopsin,6cmo,6CMO,OPSD
9,Gi/o,Class A (Rhodopsin),CB<sub>1</sub> receptor,6n4b,6N4B,CNR1


## Calculating a Feature Consensus via the Sequence Signature Tool

In [8]:
def extract_per_attr(data, attr, name):
    '''Return elements of data for which attr is equal to name'''
    return [elem for elem in data if elem[attr] == name]
    

signatures = []
for rec_class in rec_classes:
    class_name = rec_class.name
    data_per_class = None
    data_per_class = extract_per_attr(complexes, 'rec_class', class_name)

    if data_per_class:
        
        for gprot in sets.keys():
            data_per_gprot = None
            data_per_gprot = extract_per_attr(data_per_class, 'gprot', gprot)
            
            if data_per_gprot:
        
                pos_set = [elem['rec_obj'] for elem in data_per_gprot]

                signature = SequenceSignature()
                signature.setup_alignments(segments, pos_set)
                signature.calculate_signature_onesided()

                signatures.append({
                    'rec_class': class_name,
                    'gprot': gprot,
                    'signature': signature,
                })


In [9]:
signatures

[{'gprot': 'Gs',
  'rec_class': 'Class A (Rhodopsin)',
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7f2fcf580390>},
 {'gprot': 'Gi/o',
  'rec_class': 'Class A (Rhodopsin)',
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7f2fe123b048>},
 {'gprot': 'Gs',
  'rec_class': 'Class B1 (Secretin)',
  'signature': <seqsign.sequence_signature.SequenceSignature at 0x7f2fce0b7f60>}]

In [28]:
for signature_dict in signatures:
    signature = signature_dict['signature']

    sig_data = signature.prepare_display_data()
    gn = get_generic_numbers(sig_data)
    gn_flat = list(chain.from_iterable(gn))
            
    signature_dict['consensus'] = get_signature_consensus(sig_data, gn_flat)

In [27]:
{'{} - {}'.format(sig['rec_class'], sig['gprot']): len(sig['consensus']) for sig in signatures}

{'Class A (Rhodopsin) - Gi/o': 380,
 'Class A (Rhodopsin) - Gs': 292,
 'Class B1 (Secretin) - Gs': 438}

## Calculating a Feature Consensus via the Interaction Interface Matrix